In [1]:
import os

In [2]:
%pwd             # pwd->project working directory

'f:\\nlp\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\nlp\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config_filepath = Path(config_filepath)
        self.params_filepath = Path(params_filepath)
        
        # ✅ First, load YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        
        create_directories([self.config.artifacts_root], verbose=True)



    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir],verbose=True)

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config



In [9]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

                  
 

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None

        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [11]:
try:
    # create config manager
    config = ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    # create data ingestion object
    data_ingestion=DataIngestion(config=data_ingestion_config)
    # download data
    data_ingestion.download_file()
    # extract zip file
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-07-27 00:49:45,088:INFO:yaml file: config\config.yaml loaded sucessfully]
[2025-07-27 00:49:45,091:INFO:yaml file: params.yaml loaded sucessfully]
[2025-07-27 00:49:45,093:INFO:created directory at: artifacts]
[2025-07-27 00:49:45,093:INFO:created directory at: artifacts/data_ingestion]
[2025-07-27 00:49:49,165:INFO:artifacts/data_ingestion/summarizer-data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E7B8:2EFB3D:8A806:141BB1:68852A53
Accept-Ranges: bytes
Date: Sat, 26 Jul 2025 19:19:48 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10223-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1753557588.06747